In [125]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
import pickle as pkl
import matplotlib.pyplot as plt
import random
import pyfixest as pf
import statsmodels.api as sm

# Create Dataset

In [126]:
import pandas as pd
import numpy as np
from countrycode import countrycode as cc

In [127]:
gdp_data = pd.read_csv("../data/GDP_per_capita/worldbank_wdi_gdp_per_capita.csv")
tfp_data = pd.read_csv("../data/TFP/AgTFPInternational2021_AG_TFP.csv", header=2)

In [128]:
def format_target_data(data, years, year_column_format, country_column, output_var):
    formatted_outcome_var = {}
    for row in data.iterrows():
        row = row[1]
        country = row[country_column]
        formatted_outcome_var[country] = {}
        for year in years:
            year_data = float(row[year_column_format.replace("year",str(year))])
            last_year_data = float(row[year_column_format.replace("year",str(year-1))])
            formatted_outcome_var[country][year] = {}
            if np.isnan(year_data) or np.isnan(last_year_data):
                formatted_outcome_var[country][year][output_var] = np.NaN
            else:
                outcome = np.log(float(year_data)) - np.log(float(last_year_data))
                formatted_outcome_var[country][year][output_var] = outcome
    return formatted_outcome_var

In [149]:
def add_climate_vars_to_dataset(dataset, climate_val, prev_climate_val, country, year, climate_var, weights):
    dataset[country][year][f"{climate_var}_{weights}"] = climate_val
    dataset[country][year][f"{climate_var}_{weights}_2"] = np.square(climate_val)
    dataset[country][year][f"{climate_var}_{weights}_3"] = np.power(climate_val,3)
    if prev_climate_val != None:
        dataset[country][year][f"fd_{climate_var}_{weights}"] = climate_val - prev_climate_val
        dataset[country][year][f"fd_{climate_var}_{weights}_2"] = np.square(climate_val) - np.square(prev_climate_val)
        dataset[country][year][f"fd_{climate_var}_{weights}_3"] = np.power(climate_val,3) - np.power(prev_climate_val,3)
    return dataset

In [148]:
np.square(86.3568301049033) - np.square(46.8379642648779)

5263.707209289153

In [129]:
gdp_years = range(1961,2024)
tfp_years = range(1962,2022)

In [130]:
gdp_data = gdp_data.replace("..", np.NaN)
formatted_gdp_data = format_target_data(gdp_data, gdp_years, "year [YRyear]", "Country Code", "fd_ln_gdp")
formatted_tfp_data = format_target_data(tfp_data, tfp_years, "year", "ISO3", "fd_ln_tfp")

In [153]:
for climate_var in ["temp","precip","humidity"]:
    for weights in ["unweighted", "pop_weighted","ag_weighted"]:
        aggregate_var = "mean"
        if weights != "unweighted":
            aggregate_var = "weighted_mean"
        weights_no_dash = weights.replace("_","")
        data = pd.read_csv(f"../data/{climate_var}/monthly/processed_by_country/{weights}/{climate_var}.monthly.bycountry.{weights_no_dash}.mean.csv")
        for row in data.iterrows():
            prev_climate_val = None
            row = row[1]
            country = cc(row.country, origin="fips", destination="iso3c")
            if country != None:
                for year in range(1960,2024):
                    monthly_climate_vals = []
                    for month in range(1,13):
                        if month < 10:
                            month = "0" + str(month)
                        monthly_climate_vals.append(row[f"{weights_no_dash}_by_country.{aggregate_var}.X{year}.{month}.01"])
                    annual_climate_mean = np.mean(monthly_climate_vals)
                    if climate_var == "temp":
                        # celsius to kelvin
                        annual_climate_mean = annual_climate_mean - 273.15
                    elif climate_var == "precip":
                        # precipitation rate per second to total monthly precipitation (X by approx. # of seconds in a month)
                        annual_climate_mean = annual_climate_mean * 2.628e+6
                    if year in gdp_years and country in formatted_gdp_data:
                        formatted_gdp_data = add_climate_vars_to_dataset(formatted_gdp_data, annual_climate_mean, prev_climate_val, country, year, climate_var, weights)
                    if year in tfp_years and country in formatted_tfp_data:
                        formatted_tfp_data = add_climate_vars_to_dataset(formatted_tfp_data, annual_climate_mean, prev_climate_val, country, year, climate_var, weights)
                    prev_climate_val = annual_climate_mean

In [ ]:
uw_daily_temp_dir = "../data/temp/daily/processed_by_country/unweighted/"
pw_daily_temp_dir = "../data/temp/daily/processed_by_country/pop_weighted/"
aw_daily_temp_dir = "../data/temp/daily/processed_by_country/ag_weighted/"

uw_daily_precip_dir = "../data/precip/daily/processed_by_country/unweighted/"
pw_daily_precip_dir = "../data/precip/daily/processed_by_country/pop_weighted/"
aw_daily_precip_dir = "../data/precip/daily/processed_by_country/ag_weighted/"

uw_daily_humidity_dir = "../data/humidity/daily/processed_by_country/unweighted/"
pw_daily_humidity_dir = "../data/humidity/daily/processed_by_country/pop_weighted/"
aw_daily_humidity_dir = "../data/humidity/daily/processed_by_country/ag_weighted/"

In [ ]:
data = {
    "country":[],"year":[],
    # outcome variables
    "fd_ln_GDP":[],"fd_ln_TFP":[],
    # mean annual temp
    "temp_uw_mean_annual":[],"temp2_uw_mean_annual":[],"fd_temp_uw_mean_annual":[],"fd_temp2_uw_mean_annual":[],
    # mean annual precip
    "precip_uw_mean_annual":[],"precip2_uw_mean_annual:[]","fd_precip_uw_mean_annual":[],"fd_precip2_uw_mean_annual":[],
    # mean annual humidity
    
}

In [47]:
var_list = ["tmean","tmean_sq","tmean_cu","fd_tmean","fd_tmean_sq","fd_tmean_cu","prcp","prcp_sq","prcp_cu","fd_prcp","fd_prcp_sq","fd_prcp_cu"]
permutation_list = [
    np.ones(len(var_list)),
    [0,0,0,1,1,0,0,0,0,1,1,0],
    [1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0],
]


In [48]:
headers = []
for var in var_list:
    headers.append(var)
headers.append("In-sample MSE")
with open("test_out.csv", "w") as file_output:
    writer = csv.writer(file_output)
    writer.writerow(headers)
    for permutation in permutation_list:
        vars = " + ".join([var for index, var in enumerate(var_list) if permutation[index] == 1])
        regression = pf.feols(f"fd_log_tfp ~ {vars} | ISO3 + year", data=data)
        yhat = regression.predict()
        error = np.mean(np.square(yhat-data.fd_log_tfp))
        res_row = []
        for i in permutation:
            res_row.append(i)
        res_row.append(error)
        writer.writerow(res_row)

In [37]:
regression.tidy()

,Estimate,Std. Error,t value,Pr(>|t|),2.5%,97.5%
Coefficient,,,,,,
fd_tmean,-6.475999e-03,9.191869e-03,-0.704536,0.482058,-2.462014e-02,1.166814e-02
fd_tmean_sq,-4.752681e-05,2.257485e-04,-0.210530,0.833505,-4.931395e-04,3.980859e-04
fd_prcp,1.816850e-04,5.712337e-05,3.180572,0.001746,6.892724e-05,2.944428e-04
fd_prcp_sq,-1.818987e-07,5.850870e-08,-3.108918,0.002200,-2.973910e-07,-6.640643e-08


In [38]:
regression.summary()

###

Estimation:  OLS
Dep. var.: fd_log_tfp, Fixed effects: ISO3+year
Inference:  CRV1
Observations:  9255

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| fd_tmean      |     -0.006 |        0.009 |    -0.705 |      0.482 | -0.025 |   0.012 |
| fd_tmean_sq   |     -0.000 |        0.000 |    -0.211 |      0.834 | -0.000 |   0.000 |
| fd_prcp       |      0.000 |        0.000 |     3.181 |      0.002 |  0.000 |   0.000 |
| fd_prcp_sq    |     -0.000 |        0.000 |    -3.109 |      0.002 | -0.000 |  -0.000 |
---
RMSE: 0.082 R2: 0.04 R2 Within: 0.01 


In [111]:
centered_data = pf.estimation.demean(
    np.array(data[["fd_tmean", "fd_tmean_sq", "fd_prcp", "fd_prcp_sq"]]), 
    np.array(data[["encoded_iso_id","year"]]), 
    np.ones(len(data))
)

In [118]:
x = centered_data[0]
y = np.array(data.fd_log_tfp)
model = sm.OLS(y,x)
results = model.fit()
print(results.params)

[-6.47599918e-03 -4.75268083e-05  1.81684996e-04 -1.81898741e-07]


In [137]:
# predict with fixed effects
yhat = regression.predict()
error = np.square(yhat-data.fd_log_tfp)
np.mean(error)

0.006735429878711787

In [140]:
# predict without fixed effects
yhat_ = results.predict()
error = np.square(yhat_-data.fd_log_tfp)
np.mean(error)

0.006980271648325952